<a href="https://colab.research.google.com/github/andreamaggetto40/andreamaggetto40/blob/main/Quantum_Coin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Quantum Coin Simulator – Project Overview

This project is a simple yet meaningful simulation aimed at introducing the concept of **quantum superposition** through the metaphor of a coin toss.

In classical mechanics, tossing a coin yields either *heads* or *tails*, with a 50% chance for each. However, in **quantum mechanics**, a system can exist in a **superposition** of both states until a **measurement** collapses it into one.

---

##Objective

Our goal is to simulate the behavior of a quantum coin — a qubit — and visualize the probabilistic nature of its outcomes after repeated measurements.

We model the quantum coin's state as:

$$
|\psi\rangle = \alpha |0\rangle + \beta |1\rangle
$$

where:

- $|0\rangle$ corresponds to *heads*  
- $|1\rangle$ corresponds to *tails*  
- $|\alpha|^2$ and $|\beta|^2$ are the probabilities of measuring heads and tails respectively, and they always satisfy (By the definition itself of probability):

$$
|\alpha|^2 + |\beta|^2 = 1
$$

---

##Methodology

In this notebook, we:

1. Generate a quantum state by randomly assigning values to $|\alpha|$ and $|\beta|$ , following the Bloch sphere representation.
2. Perform **n = 100** measurements to simulate the coin tosses.
3. Visualize the empirical probability distribution using histograms and density plots.

---

##Purpose

This project bridges **quantum theory** and **interactive coding**, aiming to make abstract principles more tangible through code and data visualization.


In [ ]:
!pip install gradio qutip --quiet

In [ ]:
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import random
from qutip import Bloch
import tempfile

In [ ]:
# Alfa and Beta values, normalized
def generate_superposition():
    alpha = np.random.rand() + 1j * np.random.rand()
    beta = np.random.rand() + 1j * np.random.rand()
    norm_val = np.sqrt(np.abs(alpha)**2 + np.abs(beta)**2)
    alpha /= norm_val
    beta /= norm_val
    return alpha, beta

# Main Function
def simulate(tosses):
    alpha, beta = generate_superposition()
    head_prob = np.abs(alpha)**2
    tail_prob = np.abs(beta)**2

    results = ['Head' if random.random() < head_prob else 'Tail' for _ in range(tosses)]
    head_count = results.count('Head')
    tail_count = results.count('Tail')

    # 1. Probability Distribution Plot
    fig, ax = plt.subplots()
    ax.bar(["Head", "Tail"], [head_count, tail_count], color=["blue", "red"], alpha=0.6)
    x = np.linspace(0, tosses, 1000)
    mu = tosses * head_prob
    sigma = np.sqrt(tosses * head_prob * tail_prob)
    y = norm.pdf(x, mu, sigma)
    y_scaled = y * tosses / max(y)
    ax.plot(x, y_scaled, 'g--', label="Theoretical Gaussian")
    ax.set_title("Quantum Coin Toss Results")
    ax.set_ylabel("Count")
    ax.legend()

    # 2. Bloch Sphere
    psi = np.array([alpha, beta])
    bloch_vector = [2*np.real(np.conj(psi[0])*psi[1]),
                    2*np.imag(np.conj(psi[0])*psi[1]),
                    np.abs(psi[0])**2 - np.abs(psi[1])**2]

    b = Bloch()
    b.add_vectors(bloch_vector)
    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmpfile:
        b.save(tmpfile.name)
        bloch_path = tmpfile.name

    # 3. Text Output
    alpha_str = f"α = {alpha.real:.3f} + {alpha.imag:.3f}i"
    beta_str = f"β = {beta.real:.3f} + {beta.imag:.3f}i"
    ket_str = f"|ψ⟩ = ({alpha.real:.2f} + {alpha.imag:.2f}i)|0⟩ + ({beta.real:.2f} + {beta.imag:.2f}i)|1⟩"
    probs_str = f"Head Probability = {head_prob:.2%}, Tail Probability = {tail_prob:.2%}"

    return fig, bloch_path, f"{alpha_str}\n{beta_str}", ket_str, probs_str

In [ ]:
#Gradio Interface
demo = gr.Interface(
    fn=simulate,
    inputs=gr.Slider(10, 1000, value=100, label="Number of Tosses"),
    outputs=[
        gr.Plot(label="Toss Result Distribution"),
        gr.Image(type="filepath", label="Bloch Sphere"),
        gr.Text(label="α and β amplitudes"),
        gr.Text(label="State Vector |ψ⟩"),
        gr.Text(label="Probabilities (%)"),
    ],
    title="Quantum Coin Toss Simulator",
    description="Simulates a qubit in superposition (α|0⟩ + β|1⟩), visualizes measurement outcomes and Bloch sphere."
)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f1c0d06b5255457145.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
